In [74]:
import pymysql
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

config = {
    'host': 'analquick.cogfzn9i6pkm.ap-northeast-2.rds.amazonaws.com',
    'dbname': 'quicket_log',
    'user': 'lucas.kim',
    'password': 'bvnklnowiebvxo#',
    'port': '3306'
}

connection = pymysql.connect(host=config['host'],
                             user=config['user'],
                             password=config['password'],
                             db=config['dbname'],
                             charset='utf8',
                             cursorclass=pymysql.cursors.DictCursor)

In [75]:
# 쿼리 시작/종료일
start_date = '2018-01-01'
end_date = '2018-01-31'

# string -> datetime
start_date = datetime.strptime(start_date, "%Y-%m-%d")
end_date = datetime.strptime(end_date, "%Y-%m-%d")

# 산출할 쿼리
query = """

select distinct channel_id 
from tb_message
where update_time between %(query_since)s and %(query_until)s

"""


In [76]:
# 결과 저장할 데이터 프레임 만들기
result = pd.DataFrame(columns=['date', 'channel_id'])

while start_date <= end_date:
    
    query_date = datetime.strftime(start_date, "%Y-%m-%d") # datetime -> string
    query_since = query_date + ' '+ '00:00:00'
    query_until = query_date + ' '+ '23:59:59'
    
    query_result['date'] = query_date # 쿼리 실행 일자를 저장
    
    # 하루 단위로 끊어서 쿼리 실행
    query_params = {'query_since': query_since, 'query_until': query_until}
    query_result['channel_id'] = pd.read_sql(query, connection, params=query_params) # 쿼리 결과를 저장
    
    result = result.append(query_result) # 쿼리 결과를 데이터 프레임에 추가
    
    query_date = datetime.strptime(query_date, "%Y-%m-%d")  # string -> datetime
    start_date = query_date + relativedelta(days=1) # 하루 씩 증가 시킴
    

result['month'] = result['date'].str[:7] # YYYY-MM-DD -> YYYY-MM

# aggregate by month
result_monthly = result.groupby(['month'],as_index=False).agg({'channel_id': pd.Series.nunique})

result_monthly

2018-01-01
2018-01-02
2018-01-03
2018-01-04
2018-01-05
2018-01-06
2018-01-07
2018-01-08
2018-01-09
2018-01-10
2018-01-11
2018-01-12
2018-01-13
2018-01-14
2018-01-15
2018-01-16
2018-01-17
2018-01-18
2018-01-19
2018-01-20
2018-01-21
2018-01-22
2018-01-23
2018-01-24
2018-01-25
2018-01-26
2018-01-27
2018-01-28
2018-01-29
2018-01-30
2018-01-31


,month,channel_id
0,2018-01,2983424.0
